In [1]:
import os
import requests

header = {"Authorization": "Bearer "+os.getenv("ECOPORTAL_TOKEN"),
          "Content-Type": "application/json"}

api_url = ' https://data.eco-platform.org/resource/processes?search=true&distributed=true&virtual=true&metaDataOnly=false&validUntil=2022&format=JSON'

response = requests.get(url=api_url, headers=header)
main = response.json()

TypeError: can only concatenate str (not "NoneType") to str

In [1]:
def find_value(d, value, path=""):
    found = []

    if isinstance(d, dict):
        for key, dvalue in d.items():
            new_path = f"{path}.{key}" if path else key
          
            if isinstance(dvalue, str) and value in dvalue: #added for partial match
                found.append(new_path)
            elif isinstance(dvalue, (dict, list)):
                found.extend(find_value(dvalue, value, new_path))

    elif isinstance(d, list):
        for index, item in enumerate(d):
            new_path = f"{path}[{index}]"

            if isinstance(item, str) and value in item:
                found.append(new_path)  #partial
            elif isinstance(item, (dict, list)):
                found.extend(find_value(item, value, new_path))

    return found

from modules.ECOplatform.utilities import find_value

#result = await find_value(x,y,z)

In [3]:
def get_nested_value(d, keys, default='Value Not Found'):

    for key in keys:
        try:
            d = d[key]
        except (KeyError, IndexError, TypeError):
            return default
    return d

In [4]:
import re

def extract(path, type):
    if not path:
        return None
    
    if type == 1:
        pattern = r"LCIAResults\.LCIAResult\[(\d+)\]"
        match = re.search(pattern, path)
        
        if match:
            return int(match.group(1))
        else:
            return None
        
    elif type == 2:
        #pattern = r"anies\[(\d+)\]"
        pattern = r"\[(\d+)\]\.module"
        match = re.search(pattern, path)

        if match:
            return int(match.group(1))
        else:
            return None
    
    elif type != 1 and type != 2:
        print('Expected type to be 1 or 2')

In [5]:
def assign(path, type): 
    def safe_extract(d, key):
        result = find_value(d, key)
        if result and len(result) > 0:
            extracted_index = extract(result[0], 2)
            if extracted_index is not None:
                return d[extracted_index]['value']
        return None

    if type != 1:
        A2 = safe_extract(path, 'A2')
        A3 = safe_extract(path, 'A3')
        
    A4 = safe_extract(path, 'A4')
    A5 = safe_extract(path, 'A5')
    C1 = safe_extract(path, 'C1')
    C2 = safe_extract(path, 'C2')
    C3 = safe_extract(path, 'C3')
    C4 = safe_extract(path, 'C4')
    D = safe_extract(path, 'D')
    D1 = safe_extract(path, 'D1')
    D2 = safe_extract(path, 'D2')

    if type != 1:
        return A2, A3, A4, A5, C1, C2, C3, C4, D, D1, D2
    else:
        return A4, A5, C1, C2, C3, C4, D, D1, D2


In [6]:
import re

category = ['GWP-total','GWP-fossil']
#category = ['GWP-total','GWP-biogenic','GWP-fossil','GWP-IOBC']

EPDs = []
none = "Unmapped" # TODO: Return None



for x in range(10,13):
    done = 0
    path1 = 0
    path2 = 0
    num = x
    uuid = main['data'][num]['uuid']
    version = main['data'][num]['version']
    uri = main['data'][num]['uri']
    api_url = f'{uri}&format=JSON'

    response = requests.get(url=api_url, headers=header)


    # TODO: Collect "parsed" JSON in one DB, collect failures in another DB
    try:
        results = response.json()
    except requests.exceptions.JSONDecodeError as e:
        out = f'json #{num}: Failed to parse JSON. Error: {e}'
        print(out)
        print(f'Raw response text: {response.text[:200]}')
        EPDs.append(out)
        continue

    if find_value(results, 'GWP-total'):
        type = 1
    elif find_value(results, 'Global warming potential (GWP)'):
        type = 2
        print('type 2')
        EPDs.append(type)
        continue
    else:
        type = 'Unknown'
        print('unrecognized format')
        EPDs.append(type)
        continue

    ############################### DATA PATHS #################################
    EPDname = results['processInformation']['dataSetInformation']['name']['baseName'][0]['value']
    
    desc1 = get_nested_value(results, ['processInformation', 'technology', 'technologicalApplicability', 0, 'value'])
    technology_desc = get_nested_value(results, ['processInformation', 'technology', 'technologyDescriptionAndIncludedProcesses', 0, 'value'])
    desc = f"{desc1} — {technology_desc}" if desc1 and technology_desc else 'No Secondary Description'


    #comm = get_nested_value(results['processInformation']['dataSetInformation']['generalComment'][0]['value'])
    #valid = get_nested_value(results['processInformation']['time']['timeRepresentativenessDescription'][0]['value'])

    comm = get_nested_value(results, ['processInformation', 'dataSetInformation', 'generalComment', 0, 'value'])
    valid = get_nested_value(results, ['processInformation', 'time', 'timeRepresentativenessDescription', 0, 'value'])
    

    #Determine formatting style with find function   

    
    
    
    path = results['LCIAResults']['LCIAResult'][path1]['other']['anies']


    if type == 1:

        for z in range(0,2):
            find = category[z]
            find_result = find_value(results, find)

            if find_result and len(find_result) > 0:
                path1 = extract(find_result[0], 1)
            else:
                print(f"Warning: '{find}' not found in json {x}")
                continue

            #path1 = extract(find_value(results, find)[0], 1)
            #print(category[x])
            if find_value(path, 'A1-A3'):
                #print('Bundled')
                path2 = extract(find_value(path, 'A1-A3')[0], 2)
                #A11 = A12 = A13 = results['LCIAResults']['LCIAResult'][path1]['other']['anies'][path2]['value']
                if z == 0:
                    A11 = A12 = A13 = results['LCIAResults']['LCIAResult'][path1]['other']['anies'][path2]['value']
                    A14, A15, C11, C12, C13, C14, D1, D11, D12 = assign(results['LCIAResults']['LCIAResult'][path1]['other']['anies'], 1)
                elif z == 1:
                    A21 = A22 = A23 = results['LCIAResults']['LCIAResult'][path1]['other']['anies'][path2]['value']
                    A24, A25, C21, C22, C23, C24, D2, D21, D22 = assign(results['LCIAResults']['LCIAResult'][path1]['other']['anies'], 1)
                elif z == 2:
                    A31 = A32 = A33 = results['LCIAResults']['LCIAResult'][path1]['other']['anies'][path2]['value']
                    A34, A35, C31, C32, C33, C34, D3, D31, D32 = assign(results['LCIAResults']['LCIAResult'][path1]['other']['anies'], 1)
                elif z == 3:
                    A41 = A42 = A43 = results['LCIAResults']['LCIAResult'][path1]['other']['anies'][path2]['value']
                    A44, A45, C41, C42, C43, C44, D4, D41, D42 = assign(results['LCIAResults']['LCIAResult'][path1]['other']['anies'], 1)


            elif find_value(results['LCIAResults']['LCIAResult'][path1]['other']['anies'], 'A1'):
                #print('Individual')
                path2 = extract(find_value(path, 'A1')[0], 2)
                #A11 = results['LCIAResults']['LCIAResult'][path1]['other']['anies'][path2]['value']
                #A12, A13, A14, A15, C11, C12, C13, C14, D1, D11, D12 = assign(results['LCIAResults']['LCIAResult'][path1]['other']['anies'], 0)
                if z == 0:
                    A11 = results['LCIAResults']['LCIAResult'][path1]['other']['anies'][path2]['value']
                    A12, A13, A14, A15, C11, C12, C13, C14, D1, D11, D12 = assign(results['LCIAResults']['LCIAResult'][path1]['other']['anies'], 0)
                    A13Sumt = A11 + A12 + A13
                elif z == 1:
                    A21 = results['LCIAResults']['LCIAResult'][path1]['other']['anies'][path2]['value']
                    A22, A23, A24, A25, C21, C22, C23, C24, D2, D21, D22 = assign(results['LCIAResults']['LCIAResult'][path1]['other']['anies'], 0)
                    A13Sumf = A21 + A22 + A23

                    ################################## OLD PATHS ################################
                #elif z == 2:
                    #A31 = results['LCIAResults']['LCIAResult'][path1]['other']['anies'][path2]['value']
                    #A32, A33, A34, A35, C31, C32, C33, C34, D3, D31, D32 = assign(results['LCIAResults']['LCIAResult'][path1]['other']['anies'], 0)
                #elif z == 3:
                    #A41 = results['LCIAResults']['LCIAResult'][path1]['other']['anies'][path2]['value']
                    #A42, A43, A44, A45, C41, C42, C43, C44, D4, D41, D42 = assign(results['LCIAResults']['LCIAResult'][path1]['other']['anies'], 0)

                    ##############################################################################

            else:
                print('error')

    #A = results['LCIAResults']['LCIAResult'][gwpT]['other']['anies'][0]['value']



    material_data = {
    "material_id": uuid,
    "validity": {
      "gte": "2023-06-01T00:00:00",
      "lte": valid
    },
    "display_name": EPDname,
    "source": "Product Specific EPDs",
    "declared_unit": none,
    "A4_transportation": {
      "mode": none,
      "distance": {
        "qty": none,
        "unit": "km"
      }
    },
    "service_life": none,
    "waste_rate": none,
    "density": {
      "qty": none,
      "unit": "kg/m3"
    },
    "item_mass": {
      "qty": 1.0,
      "unit": "kg"
    },
    "impacts": {
      "EN15804": {
        "gwp_total": {
          "A1_A3": A13Sumt,
          "C2": C12,
          "C3": C13,
          "C4": C14,
          "D1": D11
        },
        "gwp_fossil": {
          "A1_A3": A13Sumf,
          "C2": C22,
          "C3": C23,
          "C4": C24,
          "D1": D21
        }
      }
    },
    "characteristics": {}
  }

    
    
    EPDs.append(material_data)
    print(f'json #{x} is done')



json #10 is done
json #11 is done
type 2


In [34]:
EPDs[0]

{'material_id': '7f73caec-8a20-41d2-9b7d-16c368bfe27e',
 'display_name': ' Birsta S N2 W4 c/c4  m',
 'source': 'Unmapped',
 'description': 'Birsta S - where S stands for "snow resistant" - is a railing that\nis specially manufactured to cope with close contact with snow ploughs.\nIt can also be advantageously used as collision protection\nat e.g. car parks, industrial areas and warehouse buildings. — Value Not Found,',
 'comments': 'Data quality:\nSpecific data for the product composition are provided by the manufacturer. The data represent the production of the declared product and were collected for EPD development in the year of study. Background data is based on EPDs according to EN 15804 and different LCA databases. The data quality of the raw materials in A1 is presented in the table below.\n\nAllocation:\nThe allocation is made in accordance with the provisions of EN 15804. Incoming energy and water and waste production in-house is allocated equally among all products through ma

In [ ]:
    material_data = {
        "material_id": uuid,
        "display_name": EPDname,
        "source": none,
        "description": f'{desc},',
        "comments": comm,
        "validity": valid,
        "thickness": {
            "qty": none,
            "unit": none
        },
        "service_life": none,
        "waste_rate": none,
        "transportation": {
            "mode": none,
            "distance": {
                "qty": none,
                "unit": none
            }
        }, # If density is unmapped, return "density" : None, 
        "density": {
            "qty": none,
            "unit": none
        },
        "linear_density": {
            "qty": none,
            "unit": none
        },
        "declared_unit": none,
        "lcia_method": none,
        "background_database": none,
        "impacts": {
            "gwp_total": {
                "A1": A11, "A2": A12, "A3": A13, "A4": A14, "A5": A15, "C1": C11,
                "C2": C12, "C3": C13, "C4": C14, "D": D1, "D1": D11, "D2": D12
            },
            "gwp_biogenic": {
                "A1": A21, "A2": A22, "A3": A23, "A4": A24, "A5": A25, "C1": C21,
                "C2": C22, "C3": C23, "C4": C24, "D": D2, "D1": D21, "D2": D22
            },
            "gwp_fossil": {
                "A1": A31, "A2": A32, "A3": A33, "A4": A34, "A5": A35, "C1": C31,
                "C2": C32, "C3": C33, "C4": C34, "D": D3, "D1": D31, "D2": D32
            },
            "gwp_iobc": {
                "A1": A41, "A2": A42, "A3": A43, "A4": A44, "A5": A45, "C1": C41,
                "C2": C42, "C3": C43, "C4": C44, "D": D4, "D1": D41, "D2": D42
            }
        },
        "characteristics": {}
    }